In [ ]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as dt
import seaborn as sns
import pandas as pd
from os import listdir
pd.options.display.float_format = '{:.4f}'.format
from geopy import distance
from geopy import Point
import geopandas
import shapely
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
# round to .1 so can round the magnitudes to get number earthquakes with a magnitude
def mytenths(mag):
    return round(.1 * math.floor(float(mag)/.1 + .00000001), 2)

In [ ]:
# read back in Israel data that was saved to disk
data_dir = "C:\\Users\\User\\Debbie\\Data\\"
file_path = data_dir + "output\\israel_shocks.csv"
fileToRead = open(file_path, mode='r')
isdr = pd.read_csv(fileToRead)
fileToRead.close()
isdr['datetime'] = pd.to_datetime(isdr['datetime'])
ismain = isdr[isdr['shocks']=='S'][['region','mag','datetime']]
ismain

In [ ]:
ismain['year'] = ismain['datetime'].dt.year
ismain

In [ ]:
ismain.groupby(['region','year'])['mag'].count()

In [ ]:
regs = ['Eilat-Deep','Aragonese-Deep','Arava','E.Mediter.Sea','Cyprus','Dead-Sea-Basin','Lebanon',
        'Sinai','Arnona-Dakar-Deep','Suez']

In [ ]:
# Data for a specific year represent values for the previous year or up to 50 years previous 

In [ ]:
# calculate the square root of the energy according to the power law
def energy(mag):
    return math.sqrt(pow(10,11.2+1.5*mag))

In [ ]:
# Calculates the Tn, mean Magnitude sqrt of Energy rateE over window_size number of rows
# Also calculates the magnitude to 1 decimal place for use in the regressions (power law)
# updated for the correct calculation of b for magnitudes greater than m

def feats(cl, window_size):
    cac = ismain[ismain['region']==cl].reset_index(drop=True)
    cac['tn'] = cac['datetime'].diff(periods=window_size).dt.days
    cac['meanMag'] = cac['mag'].shift(1).rolling(window_size).mean()
    cac['sqrtEnergy'] = cac['mag'].shift(1).apply(lambda x: energy(x))
    cac['rateE'] = cac['sqrtEnergy'].rolling(window_size).sum().div(cac['tn'], axis=0)
    
    # take mag to one decimal place for grouping
    cac['magg'] = cac.mag.apply(lambda x: mytenths(x))
    
    # calculate slope, intercept and mse according to least squares
    cac['a'] = 0
    cac['b'] = 0
    cac['mse'] = 0
    cac['deltaM'] = 0
        
    for i in range(0, len(cac)-window_size):  
        df = cac[i:i+window_size-1]
        x = []
        logn = [] 
        for m in df.magg:
            x.append(m)
            logn.append(math.log10(df[df['magg']>=m]['magg'].count()))
        x = np.array(x).reshape(-1,1) # take the magnitudes    
                         
        reg = LinearRegression()
        reg.fit(x, logn)
        yhat = reg.predict(x)
        cac.loc[i+window_size,'a'] = reg.intercept_
        cac.loc[i+window_size,'b'] = reg.coef_[0]
        cac.loc[i+window_size,'mse'] = mean_squared_error(logn, yhat) 
    cac.deltaM = abs(cac.a/cac.b)
    return cac

In [ ]:
# creates yearly data and calculates the MA1-MA10 and PR(MA1)-PR(MA10) and the x6 rolling averages
def cayr(df):
    df1 = df.resample('Y', on='datetime').last()
    df2 = df1.join(df.groupby('year')['mag'].max(), on='year',rsuffix='_max')
    df3 = df2.join(df.groupby('year')['mag'].count(), on='year',rsuffix='_counts') 
    df3['year'] = df3.index.year
    df3[['mag_counts']] = df3[['mag_counts']].fillna(value=0)
    df3[['mag_max']] = df3[['mag_max']].fillna(value=0)

    # Calculate the moving averages of magnitude counts
    df3['ma1'] = df3['mag_counts'].shift(1).rolling(1).mean()
    df3['ma2'] = df3['mag_counts'].shift(1).rolling(2).mean()
    df3['ma3'] = df3['mag_counts'].shift(1).rolling(3).mean()
    df3['ma4'] = df3['mag_counts'].shift(1).rolling(4).mean()
    df3['ma5'] = df3['mag_counts'].shift(1).rolling(5).mean()
    df3['ma6'] = df3['mag_counts'].shift(1).rolling(6).mean()
    df3['ma7'] = df3['mag_counts'].shift(1).rolling(7).mean()
    df3['ma8'] = df3['mag_counts'].shift(1).rolling(8).mean()
    df3['ma9'] = df3['mag_counts'].shift(1).rolling(9).mean()
    df3['ma10'] = df3['mag_counts'].shift(1).rolling(10).mean()
    
    
    # Calculate median of maximum yearly magnitudes for testing data
    med_mag = df3[df3.year<2004].mag_max.median()
    print('median maximum yearly mag: ', med_mag)
    
    # Calculate the probability that the maximum magnitude of n events recorded in the forecasted period will exceed the threshold
    df3['prob1'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma1, axis=1)
    df3['prob2'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma2, axis=1)
    df3['prob3'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma3, axis=1)
    df3['prob4'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma4, axis=1)
    df3['prob5'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma5, axis=1)
    df3['prob6'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma6, axis=1)
    df3['prob7'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma7, axis=1)
    df3['prob8'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma8, axis=1)
    df3['prob9'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma9, axis=1)
    df3['prob10'] = df3.apply(lambda x: 1 - (1 - pow(10, x.b*(med_mag-2)))**x.ma10, axis=1)
 
    # Calculate the x6 running averages for up to the last 10 years
    df3['x6_1'] = df3['mag_max'].shift(1).rolling(1).mean()
    df3['x6_2'] = df3['mag_max'].shift(1).rolling(2).mean()
    df3['x6_3'] = df3['mag_max'].shift(1).rolling(3).mean()
    df3['x6_4'] = df3['mag_max'].shift(1).rolling(4).mean()
    df3['x6_5'] = df3['mag_max'].shift(1).rolling(5).mean()
    df3['x6_6'] = df3['mag_max'].shift(1).rolling(6).mean()
    df3['x6_7'] = df3['mag_max'].shift(1).rolling(7).mean()
    df3['x6_8'] = df3['mag_max'].shift(1).rolling(8).mean()
    df3['x6_9'] = df3['mag_max'].shift(1).rolling(9).mean()
    df3['x6_10'] = df3['mag_max'].shift(1).rolling(10).mean()
 
    # calculate the 'actual' value if the median_maximum mag is > the median mag for all data
    df3.loc[df3['mag_max'] > med_mag, 'actual'] = 1
    df3.loc[df3['mag_max'] <= med_mag,'actual'] = 0
    df3[['actual']] = df3[['actual']].fillna(value=0)
   
    return df3

In [ ]:
df = pd.DataFrame()
for cl in regs:
    df1 = feats(cl,50)
    df2 = cayr(df1)
    df = pd.concat([df,df2])  # stack the new df on the old ones

In [ ]:
file_path = data_dir + "output\\israel_feats_with_all_info.csv"
df.to_csv(file_path, encoding='utf-8', index=False)

In [ ]:
df = df.drop(['mag', 'datetime', 'sqrtEnergy', 'magg', 'a', 'mag_max'], axis=1)
len(df.columns)

In [ ]:
df.groupby('region')['year'].count()

In [ ]:
file_path = data_dir + "output\\israel_feats1.csv"
df.to_csv(file_path, encoding='utf-8', index=False)

In [ ]:
# saved this file to disk - then added in the x1-x5 and x7 data into the file